# Regiment

### Introduction:

Special thanks to: http://chrisalbon.com/ for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 51.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=591f6523ccd7abd527f0f764b665db6fe065350d6f0343446ed2d1e5b5152c95
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession, functions as f

In [5]:
spark = SparkSession.builder.appName("exercise33").getOrCreate()

### Step 2. Create the DataFrame with the following values:

In [3]:
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}

### Step 3. Assign it to a variable called regiment.
#### Don't forget to name each column

In [7]:
columns = list(raw_data.keys())
data = [[*vals] for vals in zip(*raw_data.values())]
df = spark.createDataFrame(data, columns)

df.printSchema()
df.show()

root
 |-- regiment: string (nullable = true)
 |-- company: string (nullable = true)
 |-- name: string (nullable = true)
 |-- preTestScore: long (nullable = true)
 |-- postTestScore: long (nullable = true)

+----------+-------+--------+------------+-------------+
|  regiment|company|    name|preTestScore|postTestScore|
+----------+-------+--------+------------+-------------+
|Nighthawks|    1st|  Miller|           4|           25|
|Nighthawks|    1st|Jacobson|          24|           94|
|Nighthawks|    2nd|     Ali|          31|           57|
|Nighthawks|    2nd|  Milner|           2|           62|
|  Dragoons|    1st|   Cooze|           3|           70|
|  Dragoons|    1st|   Jacon|           4|           25|
|  Dragoons|    2nd|  Ryaner|          24|           94|
|  Dragoons|    2nd|    Sone|          31|           57|
|    Scouts|    1st|   Sloan|           2|           62|
|    Scouts|    1st|   Piger|           3|           70|
|    Scouts|    2nd|   Riani|           2|           

### Step 4. What is the mean preTestScore from the regiment Nighthawks?  

In [11]:
df.groupBy("regiment").mean("preTestScore").filter(f.col("regiment") == "Nighthawks").show()

+----------+-----------------+
|  regiment|avg(preTestScore)|
+----------+-----------------+
|Nighthawks|            15.25|
+----------+-----------------+



### Step 5. Present general statistics by company

*   List item
*   List item



In [23]:
df.groupBy("company").agg(*[foo(f.col(col)) for foo in (f.count,f.mean,f.min,f.max,f.stddev) for col in ("preTestScore", "postTestScore") ]).show()

+-------+-------------------+--------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-------------------------+--------------------------+
|company|count(preTestScore)|count(postTestScore)|avg(preTestScore)|avg(postTestScore)|min(preTestScore)|min(postTestScore)|max(preTestScore)|max(postTestScore)|stddev_samp(preTestScore)|stddev_samp(postTestScore)|
+-------+-------------------+--------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-------------------------+--------------------------+
|    2nd|                  6|                   6|             15.5|              67.0|                2|                57|               31|                94|       14.652644812456213|        14.057026712644463|
|    1st|                  6|                   6|6.666666666666667|57.666666666666664|                2|                25|               2

### Step 6. What is the mean of each company's preTestScore?

In [25]:
df.groupBy("company").agg(f.mean("preTestScore")).show()

+-------+-----------------+
|company|avg(preTestScore)|
+-------+-----------------+
|    2nd|             15.5|
|    1st|6.666666666666667|
+-------+-----------------+



### Step 7. Present the mean preTestScores grouped by regiment and company

In [26]:
df.groupBy("regiment", "company").agg(f.mean("preTestScore")).show()

+----------+-------+-----------------+
|  regiment|company|avg(preTestScore)|
+----------+-------+-----------------+
|Nighthawks|    1st|             14.0|
|  Dragoons|    1st|              3.5|
|Nighthawks|    2nd|             16.5|
|  Dragoons|    2nd|             27.5|
|    Scouts|    2nd|              2.5|
|    Scouts|    1st|              2.5|
+----------+-------+-----------------+



### Step 8. Present the mean preTestScores grouped by regiment and company without heirarchical indexing

### Step 9. Group the entire dataframe by regiment and company

In [28]:
grupped_reg_company = df.groupBy("regiment", "company")

### Step 10. What is the number of observations in each regiment and company

In [30]:
grupped_reg_company.count().show()

+----------+-------+-----+
|  regiment|company|count|
+----------+-------+-----+
|Nighthawks|    1st|    2|
|  Dragoons|    1st|    2|
|Nighthawks|    2nd|    2|
|  Dragoons|    2nd|    2|
|    Scouts|    2nd|    2|
|    Scouts|    1st|    2|
+----------+-------+-----+



### Step 11. Iterate over a group and print the name and the whole data from the regiment